In [21]:
from bs4 import BeautifulSoup
import ollama

In [23]:
# Loop through each XML file
extracted_text = []
for file_number in range(1, 11):
    file_name = f"ep-{file_number}.xml"  # Replace with your actual file name
    with open(file_name, 'r', encoding='utf-8') as f:
        print(file_name)
        file_content = f.read()
        soup = BeautifulSoup(file_content, 'xml')
        
        # Find all <p> elements and extract text
        p_elements = soup.find_all('p')
        for p in p_elements:
            text = p.get_text(strip=True)  # Get text without spaces
            extracted_text.append(text)

ep-1.xml
ep-2.xml
ep-3.xml
ep-4.xml
ep-5.xml
ep-6.xml
ep-7.xml
ep-8.xml
ep-9.xml
ep-10.xml


In [24]:
import time

chunk_size = 100
iterations = 3  # Number of iterations
text_chunks = [extracted_text[i:i+chunk_size] for i in range(0, len(extracted_text), chunk_size)]

for iteration in range(iterations):
    print(f"Iteration {iteration + 1}/{iterations}")
    
    all_responses = []
    
    for idx, chunk in enumerate(text_chunks, start=1):
        combined_text = " ".join(chunk)
        
        prompt = f"""
        - Below is a subtitle file from a TV episode. Your task is to identify and extract any mentions of famous books and their corresponding authors from the text.
        - Do not include random text or phrases that are not actual book titles.
        - If you cannot find any book mentions, output 'NA'.
        - Ensure that each book title is accurately matched with its author.
        - Output the list in the format: '<Name of the book> : <Name of the author>'.
        - Separate each book entry with a newline.
        - Dont supplement with any additional notes at the start or the finish of the output text. 
        
        Conversation:
        {combined_text}
        """
        
        # Call the LLaMA 3 model
        response = ollama.chat(
            model="llama3",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Extract the response content
        response_text = response['message']['content'].strip()
        print(f"Chunk {idx}/{len(text_chunks)} Output:\n{response_text}\n")
        all_responses.append(response_text)
        
        time.sleep(10)  # Adjust as needed

    # For the next iteration, use the cleaned output from this iteration as input
    cleaned_input = "\n".join([r for r in all_responses if r != 'NA'])  # Filter out 'NA' entries
    text_chunks = [cleaned_input[i:i+chunk_size] for i in range(0, len(cleaned_input), chunk_size)]  # Re-chunk the output for the next iteration

# Final cleaned output
final_output = "\n".join(all_responses)
print("Final Cleaned Output:")
print(final_output)


Here is the output:

Huckleberry Finn : Mark Twain
Moby Dick : Herman Melville
Madame Bovary : Gustave Flaubert
The Little Match Girl : Hans Christian Andersen
Wuthering Heights : Emily Brontë
Peyton Place : Richard Peabody Simmons
Gaucho : Jorge Luis Borges
TO PERM OR NOT TO PERM : William Shakespeare (as Hamlet)
The Counter of Saint Petersburg : Alexander Pushkin
VALLEY OF THE DOLLS : Jacqueline Susann
All the King's Men : Robert Penn Warren
Mushrooms : Lewis Carroll
9 1/2 Weeks : Judith Viola
Willy Wonka : Roald Dahl
The Way We Were : Arthur Laurents
Beloved : Toni Morrison
Pride and Prejudice : Jane Austen
The Portable Dorothy Parker : Dorothy Parker
Alice in Wonderland : Lewis Carroll
The Gift of the Magi : O. Henry
Metamorphosis : Kafka
A Christmas Carol : Charles Dickens
